In [2]:
import pandas as pd, numpy as np, os
from scipy.stats import boxcox

from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
import bayes_net_utils as bn

# Intro

Notebook takes observed and predicted lake chemistry/ecol values output by notebook 02_BN_development_1Season (from the continuous network cross validation section). These are then discretized into WFD-related boundaries, and classification error is calculated. This classification error can then be compared to the classification error obtained by cross validation of discrete networks using the same classifications, or just to provide supporting error info to accompany predictions of probabilities of being in a certain class.

# Set up for processing

In [3]:
# USER INPUT

# Boundaries dictionary, copied from notebook B_seasonal_data_matrix_1Season
bound_dict = {
             'TP': [29.5], # No data below 20, so drop this class boundary. 29.5 is middle of 'Mod' class   
             'chla': [20.0],  # WFD boundaries: [10.5, 20.0]. But only 6 d.p. under 10.5 so merge G and M classes.
                              # For predicting cyano, would be better 17.4.   
             'colour': [48.0], # [51.2] if comparing CV scores of continuous and discrete. [48.0] if calculating CV score of final network (66th percentile)
             'cyano': [1.0], # M-P boundary is 2.0, but there were only 2 values in this class. Plenty above 2 tho.
             }

# Alter the boundaries in the boundaries dict for cyano, to take account of the box-cox transformation applied to the continuous
# data:  y* = (y^L - 1)/L, where we used lambda=0.1 when transforming original cyano data
bound_dict['cyano'] = boxcox(bound_dict['cyano'], lmbda=0.1)

met_source = 'metno' #'metno' or 'era5'

var_li = ['TP','chla','cyano','colour'] # What do you want to produce stats for? Need to have corresponding files in 'Data/CrossValidation/%s' %met_source folder

In [4]:
def xval_postprocess(var, fpath):
    """
    Function to read in a csv of observed and predicted values from a continuous
    Bayesian belief network produced in BNLearn R notebook, and calculate correlation
    coefficients, and then classify according to WFD and work out classification error
    for comparison to classification error calculated directly in BNLearn using discrete network.
    
    Inputs:
        var: string, one of 'TP','chla','cyano','colour_summer'
        fpath: string giving location of csv to be read in. csv should have columns:
                'obs_1','pred_1','obs_2','pred_2',... where _1, _2, etc. is the cross
                validation run number.
    Returns:
    Printed output, plus dictionary of results, with keys
        'corr_coeffs': series of correlation coefficients, one value per cross validation run
        'classification_errors': series of classification errors, one value per cross validation run
        'cont_data_dict': dict of observed and predicted dfs, continuous data, one df per xval run
        'classified_data_dict': dict of observed and predicted dfs, classified data, one df per xval run
        
    """

    # ---------------------------------------------------------------------------------
    # Read in data
    df = pd.read_csv(fpath, index_col=0)

    # ---------------------------------------------------------------------------------
    # Split into separate dataframes for each cross validation run
    cont_dict = {}  # Key: run number, returns df with obs and pred
    for i, col_name in enumerate(df.columns):
        if i % 2 == 0:  # If even, i.e. only do this for half the cols
            run_no = int(col_name.split("_", 1)[1])
            if run_no == 1:
                temp_df = df.iloc[:, [0, 1]]
            else:
                temp_df = df.iloc[:, [2 * run_no - 2, 2 * run_no - 1]]

            temp_df.columns = ["obs", "pred"]
        cont_dict[run_no] = temp_df

    # ---------------------------------------------------------------------------------
    # Calculate correlation coefficients, convert to WFD classes and work out classification error

    cc_dict = {}  # key: run_no, returns correlation coeff
    rmse_dict = {}  # key: run_no, returns mse (pred - obs)
    classified_dict = {} # Key: run number. Returns df with cols 'obs' and 'pred', discrete data
    class_error_dict = {}
    mcc_dict = {}
    roc_auc_dict = {}

    for run_no in cont_dict.keys():

        cont_df = cont_dict[run_no]  # df with continuous data

        # Correlation coefficients
        cc_dict[run_no] = cont_df["obs"].corr(cont_df["pred"], method="pearson")

        # Mean square error
        rmse = np.sqrt(np.mean(((cont_df["pred"] - cont_df["obs"]) ** 2)))
        rmse_dict[run_no] = rmse

        # Classify obs and pred into WFD (or related) class boundaries
        disc_df = pd.DataFrame(index=cont_df.index, columns=cont_df.columns)  # New empty df to be populated
        for col in cont_df.columns:
            disc_df[col] = cont_df[col].apply(lambda x: bn.discretize(bound_dict[var], x))
        classified_dict[run_no] = disc_df

        # Calculate classification error (proportion of time model predicted class correctly)
        error = bn.classification_error(disc_df['obs'],disc_df['pred'])
        class_error_dict[run_no] = error
        
        # Calculate matthew's correlation coefficient and ROC_AUC score
        mcc_dict[run_no] = matthews_corrcoef(disc_df['obs'].values, disc_df['pred'].values) 
        roc_auc_dict[run_no] = roc_auc_score(disc_df['obs'].values, disc_df['pred'].values)

    # ---------------------------------------------------------------------------------
    # Aggregate results over runs

    corr_coeffs = pd.Series(cc_dict)  # These match those calculated by bnlearn, good!
    rmses = pd.Series(rmse_dict)
    errors = pd.Series(class_error_dict)
    mccs = pd.Series(mcc_dict)
    roc_aucs = pd.Series(roc_auc_dict)

    # ---------------------------------------------------------------------------------
    # Take a look at the output
#     print("Correlation coefficients, %s:" % var)
#     print(corr_coeffs)
#     print("\nMean correlation coefficient, %s: %s" % (var, corr_coeffs.mean()))

#     print("\nmse, %s:" % var)
#     print(mses)
#     print("Mean mse, %s: %s" % (var, mses.mean()))

#     print("\nClassification errors, %s:" % var)
#     print(errors)
#     print("Mean classification error for %s: %s" % (var, errors.mean()))

    # Return dictionaries
#     results_dict = {
#                     "corr_coeffs": corr_coeffs,
#                     "classification_errors": errors,
#                     "cont_data_dict": cont_dict,
#                     "classified_data_dict": classified_dict,
#                     }
#     return results_dict
    results_series = pd.Series(data = np.array([corr_coeffs.mean(),
                                                rmses.mean(),
                                                errors.mean(),
                                                mccs.mean(),
                                                roc_aucs.mean()]
                                              ),
                               index = ['mean_CC','mean_rmse','mean_class_error',
                                       'mean_mcc', 'mean_ROC_AUC'])
    
    return results_series

# Calculate stats

In [5]:
# Get list of files to loop over. Produced in 02_BN_development_1Season

CV_obs_sim_folder = "../Data/CrossValidation/CV_predictions"
# CV_obs_sim_folder = "../Data/CrossValidation/CV_predictions_allNodes_and_metno"
fpaths = os.listdir(CV_obs_sim_folder)

fpath_dict = {}
for file in fpaths:
    var = file.split('_')[0]
    if var in var_li:
        if var in fpath_dict.keys():
            fpath_dict[var].append(file)
        else:
            fpath_dict[var] = [file]

fpath_dict

{'chla': ['chla_cont_fromPredictableNodes_era5.csv',
  'chla_cont_fromPredictableNodes_nomet.csv'],
 'cyano': ['cyano_cont_fromPredictableNodes_exclRainNode.csv',
  'cyano_cont_fromPredictableNodes_exclMetNodes.csv',
  'cyano_cont_fromPredictableNodes_nomet.csv',
  'cyano_cont_fromPredictableNodes_era5.csv',
  'cyano_cont_fromPredictableNodes_exclWindSpeedNode.csv'],
 'colour': ['colour_cont_fromPredictableNodes_era5.csv',
  'colour_cont_fromPredictableNodes_nomet.csv'],
 'TP': ['TP_cont_fromPredictableNodes_nomet.csv',
  'TP_cont_fromPredictableNodes_era5.csv']}

In [6]:
series_li = []
for var in var_li:
    for file in fpath_dict[var]:
        stats_series = xval_postprocess(var, os.path.join(CV_obs_sim_folder, file))
        stats_series.name = file.split('.')[0]
        series_li.append(stats_series)

df = pd.concat(series_li, axis=1, keys=[s.name for s in series_li]).transpose()

df['Variable'] = [i.split('_')[0] for i in list(df.index)]
df['met_included'] = [i.split('_')[-1] for i in list(df.index)]
df = df.set_index(['Variable','met_included']).sort_index()

# df['met_data'] = [i.split('_')[-1] for i in list(df.index)] # Comment out if just looking at era5 data
# df['nodes_used'] = [i.split('_')[2][4:-5] for i in list(df.index)] # Comment out if just using predictable nodes
# df['wind_yn'] = [i.split('_')[-2] for i in list(df.index)]
# df.loc[df['wind_yn']!='noWind','wind_yn'] = 'Wind'
# df = df.set_index(['Variable','met_data','nodes_used','wind_yn']).sort_index()

df = df.reset_index()

# Write to csv
df.to_csv('../Data/CrossValidation/Stats/CV_results_predictableNodes_era5-vs-nomet.csv', index=False)

df

,Variable,met_included,mean_CC,mean_rmse,mean_class_error,mean_mcc,mean_ROC_AUC
0,TP,era5,0.577181,3.915841,0.325641,0.342134,0.669444
1,TP,nomet,0.577181,3.915841,0.325641,0.342134,0.669444
2,chla,era5,0.567892,4.592084,0.276923,0.219387,0.589123
3,chla,nomet,0.529636,4.750781,0.303846,0.116415,0.544156
4,colour,era5,0.833830,9.002310,0.234615,0.461131,0.725000
5,colour,nomet,0.818836,9.360379,0.230769,0.471405,0.730769
6,cyano,era5,0.621585,0.995830,0.185417,0.636801,0.818706
7,cyano,exclMetNodes,0.677947,0.960638,0.133333,0.762738,0.876573
8,cyano,exclRainNode,0.623040,0.995993,0.189583,0.627198,0.814161
9,cyano,exclWindSpeedNode,0.666969,0.964675,0.127083,0.772473,0.882343


# TP

TP: Comparison of classification errors between continuous and discrete networks:

Including met.no wind:
- Predictable/measurable nodes, discrete BN: 0.55
- Predictable/measurable nodes, continuous BN: 0.36
- All nodes, discrete BN: 0.59
- All nodes, continuous BN: 0.32

**Other structures**

Remove met.no wind:
- All stats same or slightly improved without wind in continuous network. Therefore **decide to remove wind-TP link**.
- **Comparison of discrete and continuous network classification error, without metno wind**, excluding 2019 data, becomes:

    - Predictable/measurable nodes, discrete: 0.51
    - Predictable/measurable nodes, continuous: 0.33
    - All nodes, discrete: 0.53
    - All nodes, continuous: 0.31
    
So take this as the final comparison of discrete vs continuous for TP, and part of the justification for going continuous. Other justification is the cpts in the discrete, which don't show the right behaviour (see markdown comments in BN development notebook).

# Chl-a

In [7]:
df.loc[df['Variable']=='chla']

,Variable,met_included,mean_CC,mean_rmse,mean_class_error,mean_mcc,mean_ROC_AUC
2,chla,era5,0.567892,4.592084,0.276923,0.219387,0.589123
3,chla,nomet,0.529636,4.750781,0.303846,0.116415,0.544156


**Comparison of classification error between continuous and discrete, met.no data (data to 2018):**

Whole network used to predict chl-a node:
- discrete BN:0.08
- continuous BN: 0.21
- continuous, adding extra WFD class (G-M boundary at 10.0): 0.44

Only include nodes that will be updated when forecasting:
- discrete BN: 0.08
- continuous BN: 0.32
- continuous, adding extra WFD class (G-M boundary at 10.0): 0.49

Lower classification error when using the discrete network. But problems with the discrete chla cpts outweigh these differences:
- When chla_prevSummer is high, changing wind speed from L to H doesn't result in any change to probs for low TP. But if TP is high, higher wind speed makes it more likely to have high chla. This isn't right, and is only because of a lack of data points. Therefore would have had to remove the wind-chla link.
- Even after doing this, still have probs: when previous summer's chl-a is low, chla is always predicted to be low, and TP has no effect. When previous summer's chl-a is high, the TP effect is the wrong way around: when TP is low, expect high chla. When TP is high, have a lower chance of high TP than when TP is low. Again, just due to low data volume

**Removing wind:**

- No change or slight deterioration, depending on whether you predict using just predictable nodes (no change) or the whole network (slight deterioration).

# Cyano

In [8]:
df.loc[df['Variable']=='cyano']

,Variable,met_included,mean_CC,mean_rmse,mean_class_error,mean_mcc,mean_ROC_AUC
6,cyano,era5,0.621585,0.995830,0.185417,0.636801,0.818706
7,cyano,exclMetNodes,0.677947,0.960638,0.133333,0.762738,0.876573
8,cyano,exclRainNode,0.623040,0.995993,0.189583,0.627198,0.814161
9,cyano,exclWindSpeedNode,0.666969,0.964675,0.127083,0.772473,0.882343
10,cyano,nomet,0.674139,0.950204,0.125000,0.777620,0.884615


**Comparison of classification error, continuous vs discrete networks (met.no data, data to 2018):**

- Predictable/measurable nodes, discrete: 0.23
- Predictable/measurable nodes, continuous: 0.17
- All nodes, discrete: 0.13
- All nodes, continuous: 0.18

In this case, the continuous network has lower classification error than the discrete network when using predictable/measurable nodes, but slightly higher error when using all nodes (first time this has been the case). But errors still pretty low, and added benefit of the relationships being sensible between nodes rather than the cpt issue we have for one of the cpts in the cyano class: in the high summer colour class, as chla increases from L to H, the chance of high cyano decreases. I think we would expect positive relationship between chla and cyano regardless of colour, just a lower chance of high cyano when colour is higher. So this is another low data vol artefact.

**To investigate in the future:**
- Cross val of cyano predictive ability with/without colour-cyano link

# Proposed changes to structure based on cross validation results

(in this notebook, and the BN_Development_1Season one)

- Remove wind-TP link. Keep wind-chla link.
- Keep rain-colour link